<a href="https://colab.research.google.com/github/RehamJamal13/model/blob/main/CoAuthorWithWeights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Download pakages
#NATURAL LANGUAGE PROCESSING
!pip install pandas
!pip install networkx
!pip install matplotlib
! pip install pyvis
!pip install nameparser

In [ ]:
#@title Load Packages
#!pip install nameparser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
#from Topic_Modeling import *
#import scispacy
#import spacy
#import en_core_sci_lg
import networkx as nx
from pathlib import Path
from itertools import combinations
from collections import Counter
from itertools import chain
import random
from pyvis.network import Network
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.cluster import KMeans

In [ ]:
#@title Load Data
data= pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Copy_of_Author_paper.csv')

##CoAuthor Network

In [ ]:
#data = data.drop(columns=['Author_split'])
data['Author_split']=data['Author'].apply(lambda x: x.replace("'", "").strip('][').split(', '))



In [ ]:
data['Author_pairs']=data['Author_split'].apply(lambda x:list(combinations(x, 2)))
data.head()

,DOC_ID,Author,Author_split,Author_pairs
0,0,['Unknown author'],[Unknown author],[]
1,2,['Unknown author'],[Unknown author],[]
2,3,['Unknown author'],[Unknown author],[]
3,4,['Unknown author'],[Unknown author],[]
4,5,['Unknown author'],[Unknown author],[]


In [ ]:
data[200:210]

,DOC_ID,Author,Author_split,Author_pairs
200,390,"['a. elbasit ie khalil', ' i. f. elbashir', ' ...","[a. elbasit ie khalil, i. f. elbashir, m. i....","[(a. elbasit ie khalil, i. f. elbashir), (a. ..."
201,391,"['a. elgadir t. m. elbashir', ' m. i. berzins'...","[a. elgadir t. m. elbashir, m. i. berzins, k...","[(a. elgadir t. m. elbashir, m. i. berzins), ..."
202,392,"['a. elgadir t. m. theander', ' t. g. elghazal...","[a. elgadir t. m. theander, t. g. elghazali, ...","[(a. elgadir t. m. theander, t. g. elghazali)..."
203,394,"['a. elgayoum s. m. el-rayah', ' e. a. giha', ...","[a. elgayoum s. m. el-rayah, e. a. giha, h. a.]","[(a. elgayoum s. m. el-rayah, e. a. giha), (a..."
204,395,"['a. elgayoum s. m. el-rayah el', ' a. giha', ...","[a. elgayoum s. m. el-rayah el, a. giha, h. a.]","[(a. elgayoum s. m. el-rayah el, a. giha), (a..."
205,396,"['a. elgayoum s. m. el-rayah el', ' a. giha', ...","[a. elgayoum s. m. el-rayah el, a. giha, h. a.]","[(a. elgayoum s. m. el-rayah el, a. giha), (a..."
206,397,"['a. magid y. m. elussein', ' e. a. omer', ' m...","[a. magid y. m. elussein, e. a. omer, m. m. ...","[(a. magid y. m. elussein, e. a. omer), (a. m..."
207,398,"['a. rahman s. h. mohamedani', ' a. a. mirgani...","[a. rahman s. h. mohamedani, a. a. mirgani, ...","[(a. rahman s. h. mohamedani, a. a. mirgani),..."
208,399,"['a. s. safi', ' ahmed']","[a. s. safi, ahmed]","[(a. s. safi, ahmed)]"
209,400,"['a. wahab m. m. ismail', ' a. m. rais', ' m. ...","[a. wahab m. m. ismail, a. m. rais, m. a. is...","[(a. wahab m. m. ismail, a. m. rais), (a. wah..."


In [ ]:
ai_authors=pd.DataFrame(data['Author_split'].tolist()).rename(columns={0:'Author_split'})
papers_by_authors=ai_authors.groupby(['Author_split']).size().reset_index().rename(columns={0:'Number of Papers Published'}).sort_values("Number of Papers Published",ascending=False)
papers_by_authors.shape

(5127, 2)

In [ ]:
papers_by_authors['Number of Papers Published'].describe()

count    5127.000000
mean        2.580066
std         8.605011
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max       294.000000
Name: Number of Papers Published, dtype: float64

In [ ]:
## Keeping Authors who have published more than 2 Papers
nodes_to_keep=papers_by_authors.loc[papers_by_authors['Number of Papers Published']>2,'Author_split'].tolist()
len(nodes_to_keep)

808

#Generating the Edges of the Co-Author Network

In [ ]:
authors_pairs=data['Author_pairs'].tolist()
authors_edge_list=[item for sublist in authors_pairs for item in sublist]
authors_weighted_edge_list=list(Counter(authors_edge_list).items())
authors_weighted_edge_list=[(row[0][0],row[0][1],row[1]) for idx,row in enumerate(authors_weighted_edge_list)]
authors_weighted_edge_list[0:10]

[('-elgayoum', ' salwa. m. e. el-feki', 1),
 ('-elgayoum', ' abd el-karim ahmed. mahgoub', 1),
 ('-elgayoum', ' babiker ahmed. el-rayah', 1),
 ('-elgayoum', ' elamin. giha', 1),
 ('-elgayoum', ' hayder.  a.', 1),
 (' salwa. m. e. el-feki', ' abd el-karim ahmed. mahgoub', 1),
 (' salwa. m. e. el-feki', ' babiker ahmed. el-rayah', 1),
 (' salwa. m. e. el-feki', ' elamin. giha', 1),
 (' salwa. m. e. el-feki', ' hayder.  a.', 1),
 (' abd el-karim ahmed. mahgoub', ' babiker ahmed. el-rayah', 1)]

In [ ]:
G1=nx.Graph()
G1.add_weighted_edges_from(authors_weighted_edge_list)
print(len(G1.nodes()))

35439


In [ ]:
## From the complete Graph, create a subgraph, with only the nodes to keep
#Keeping Authors who have published at least 3 papers
sub_g=nx.subgraph(G1,nodes_to_keep)
G=nx.Graph(sub_g)
print(len(G.nodes()))
isolated_node=nx.isolates(G)
len(list(isolated_node))

806


806

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))
len(G.nodes)

0

In [ ]:
del G1, sub_g
print("Number of Nodes in Author Graph ",len(G.nodes()))
print("Number of Edges in AUthor Graph ",len(G.edges()))

Number of Nodes in Author Graph  0
Number of Edges in AUthor Graph  0


#Implementing Deep Walk

In [ ]:
def getRandomWalk(graph,node,length_of_random_walk):
    """ This function takes NetworkX Graph and a Node and generate random walk for a given length

    Returns the random walk (list of nodes traversed)

    Note: The same node may occcur more than once in a Random Walk.
    """
    start_node=node
    current_node=start_node
    random_walk=[node]
    for i in range(0,length_of_random_walk):
        ## Choose a random neighbour of the current node

        current_node_neighbours=list(graph.neighbors(current_node))
        chosen_node=random.choice(current_node_neighbours)
        current_node=chosen_node
        random_walk.append(current_node)
    return random_walk

In [ ]:

### For every Node in the Graph, get randomwalks . For eahc node, let us get random walks say around 10 times each of path length 10
num_sampling=10
random_walks=[]
length_of_random_walk=10
for node in tqdm(G.nodes(),desc="Iterating Nodes"):

    for i in range(0,num_sampling):
        random_walks.append(getRandomWalk(G,node,length_of_random_walk))


Iterating Nodes: 0it [00:00, ?it/s]

In [ ]:
deepwalk_model=Word2Vec(sentences=random_walks,window=5,sg=1,negative=5,vector_size=128,epochs=20,compute_loss=True)

RuntimeError: ignored

In [ ]:
deepwalk_model.save("deepwalk_biblography.model")

NameError: ignored

#Lets Look who are similar authors

In [ ]:
def getSimilarNodes(model,node):
    """
    This function takes deepwalk model and a node

    Returns the top 10 nodes (author) similar to the given node
    """
    similarity=model.wv.most_similar(node)
    similar_nodes=pd.DataFrame()
    similar_nodes['Similar_Node']=[row[0] for i,row in enumerate(similarity)]
    similar_nodes['Similarity_Score']=[row[1] for i,row in enumerate(similarity)]
    similar_nodes['Source_Node']=node
    return similar_nodes

In [ ]:
getSimilarNodes(deepwalk_model,"ali")

NameError: ignored

In [ ]:
ai_authors=pd.DataFrame(data['Author_split'].tolist()).rename(columns={0:'Author_split'})
papers_by_authors=ai_authors.groupby(['Author_split']).size().reset_index().rename(columns={0:'Number of Papers Published'}).sort_values("Number of Papers Published",ascending=False)
papers_by_authors

In [ ]:
def flattenList(nested_list):
    flat_list = [item for sublist in nested_list for item in sublist]
    return flat_list

In [ ]:
#Lets get the graph of these top Authors with their first step neighbours.We consider the authors ranked 4th to 10th to help visualise the clusters better
def getCoAuthorshipNetwork(graph,initial_nodes):
    """
    This function takes a Graph and list of initial nodes

    Returns the set of immediate neighbours of these nodes

    """
    total_neighbours=0
    nodes_set=[initial_nodes]
    for node in initial_nodes:
        #print(node)
        neighbours=list(graph.neighbors(node))
        total_neighbours=total_neighbours+len(neighbours)

        nodes_set.append(neighbours)
    print(total_neighbours)
    nodes_set=flattenList(nodes_set)
    return list(set(nodes_set))

In [ ]:
coauthor_nodes=getCoAuthorshipNetwork(G,papers_by_authors['Author_split'].tolist()[4:10])
print("Number of CoAuthor Nodes ",len(coauthor_nodes))

Generate a Co-Author Graph from the complete Graph

In [ ]:
coauthor_subgraph=nx.subgraph(G,coauthor_nodes)
print("number of edges in the CoAuthor Subgraph ",len(coauthor_subgraph.edges()))

In [ ]:
nx.write_gexf(coauthor_subgraph, "CoAuthor_Subgraph_Author4to10.gexf")

In [ ]:
#coauthor_subgraph=nx.read_gexf("CoAuthor_Subgraph_Top50Author.gexf")
print("number of edges in the CoAuthor Subgraph ",len(coauthor_subgraph.edges()))

In [ ]:
pyvis_nt=Network(notebook=True,height='800px', width='100%',heading='', cdn_resources='in_line')

print("Creating PyVis from NetworkX")
pyvis_nt.from_nx(coauthor_subgraph)

print("Saving PyVis Graph")
pyvis_nt.show("Author4to10_CoAuthorGraph.html")

Cluster the Nodes

In [ ]:
def getCosineDistanceMatrix(vectors):
    '''
    This function takes list of vectors or numpy array

    Returns the pairwise cosine similarity matrix
    '''
    if type(vectors)==list:
        X=np.asarray(vectors)
    elif type(vectors)==np.ndarray:
        X=vectors
    else:
        print("Error in Data Type . Need to Pass list or numpy array as input argument")
        return []
    cosine_dist=cosine_distances(X)
    return cosine_dist

In [ ]:
coauthor_nodes=list(coauthor_subgraph.nodes)
print("Number of CoAuthor Subgraph Nodes",len(coauthor_nodes))

In [ ]:
coauthor_embeddings=[deepwalk_model.wv[node] for node in coauthor_nodes]
coauthor_embeddings=np.asarray(coauthor_embeddings)
print(coauthor_embeddings.shape)
cosine_dist=getCosineDistanceMatrix(coauthor_embeddings)

In [ ]:
#K-mean for optimal number of clusters
sse=[]
k_list=[]
for k in range(2,20):

    km=KMeans(n_clusters=k)
    km.fit(cosine_dist)
    sse.append(km.inertia_)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot([i for i in range(2,20)],sse)

In [ ]:
#from figure lets pick 8 clusters
km=KMeans(n_clusters=8)
coauthor_clusters=km.fit_predict(cosine_dist)
coauthor_cluster_dict={node:str(coauthor_clusters[idx]) for idx,node in enumerate(coauthor_nodes)}
nx.set_node_attributes(coauthor_subgraph,coauthor_cluster_dict,"group")

In [ ]:
pyvis_nt=Network(notebook=True,height='600px', width='100%',heading='Author Network', cdn_resources='in_line')

print("Creating PyVis from NetworkX")
pyvis_nt.from_nx(coauthor_subgraph)
pyvis_nt.toggle_physics(True)
print("Saving PyVis Graph")
#pyvis_nt.show_buttons()
#pyvis_nt.set_options('var options = {"edges": { "color": { "inherit": true },"smooth": false},"physics": {"hierarchicalRepulsion": { "centralGravity": 0 },"minVelocity": 0.75, "solver": "hierarchicalRepulsion","timestep": 0.18}}')


pyvis_nt.show("Author4to10_CoAuthorGraph_Clustered.html")


Visualising El hassan Network

In [ ]:
el_hassan_nodes=getCoAuthorshipNetwork(G,['el hassan'])
el_hassan_network=nx.subgraph(G,el_hassan_nodes)
print("Number of Nodes in El hassan Network ",len(el_hassan_network.nodes()))
print("Number of Edges in El hassan Network ",len(el_hassan_network.edges()))

In [ ]:
el_hassan_nt=Network(notebook=True,height='800px', width='100%',heading='El hassan Network', cdn_resources='in_line')

print("Creating PyVis from NetworkX")
el_hassan_nt.from_nx(el_hassan_network)
el_hassan_nt.toggle_physics(True)
#el_hassan_nt.enable_physics(True)
print("Saving PyVis Graph")

el_hassan_nt.show_buttons()
el_hassan_nt.show("El hassan_CoAuthorGraph.html")